In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
data = pd.read_csv("/kaggle/input/heart-failure-prediction/heart.csv")

In [3]:
data.head()

In [4]:
data.shape

In [5]:
data = data.drop_duplicates(keep='first')
data.shape

### Label encoding for changing string value to Numeric

In [6]:
from sklearn.preprocessing import LabelEncoder
data.head(2)

In [7]:
li = ['Sex','ChestPainType','RestingECG','ExerciseAngina','ST_Slope']
for i in li:
    data[i] = LabelEncoder().fit_transform(data[i])

### Find Correlation among Data 

In [8]:
sns.heatmap(data.corr(),annot=True,fmt ='.1f',mask = np.triu(data.corr()))

In [9]:
import warnings
warnings.filterwarnings('ignore')

# visualizationS

In [10]:
plt.figure(figsize=(15,15))
for i in enumerate(data.columns):
    plt.subplot(3,4,i[0]+1)
    sns.distplot(data[i[1]])
    

In [11]:
plt.figure(figsize=(15,15))
for i in enumerate(data.columns):
    plt.subplot(3,4,i[0]+1)
    sns.boxplot(data[i[1]])
    

In [12]:
def remove(data):
  Q1 = np.percentile(data, 25, interpolation = 'midpoint')
  
# Third quartile (Q3)
  Q3 = np.percentile(data, 75, interpolation = 'midpoint')
  iqr = Q3 - Q1
  percentile25 = data.quantile(0.25)
  percentile75 = data.quantile(0.75)

  upper_limit = percentile75 + 1.5 * iqr
  lower_limit = percentile25 - 1.5 * iqr
  data = np.where(
    data > upper_limit,upper_limit,
    np.where(
        data < lower_limit,
        lower_limit,
        data
    )
  )
  return data

#feature[feature['discount_percent'] > upper_limit]
#feature[feature['discount_percent'] < lower_limit]



In [13]:
data.columns

In [14]:
li =['RestingBP','Cholesterol','MaxHR','Oldpeak']
for i in li:
    data[i]=remove(data[i])

In [15]:
plt.figure(figsize=(15,15))
for i in enumerate(data.columns):
    plt.subplot(3,4,i[0]+1)
    sns.boxplot(data[i[1]])
    

In [16]:
plt.figure(figsize=(15,15))
for i in enumerate(data.columns):
    plt.subplot(3,4,i[0]+1)
    sns.distplot(data[i[1]])
    

In [17]:
feature = data.drop('HeartDisease',axis=1)
label = data['HeartDisease']

# Splitting Data

In [18]:
from sklearn.model_selection import train_test_split
trainF,testF,trainL,testL = train_test_split(feature,label,random_state=42,test_size=.2)

# choosing model ,train, test, evalute

In [19]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(trainF,trainL)
pred = model.predict(testF)

In [20]:
from sklearn.metrics import classification_report
print(classification_report(pred,testL))

In [21]:
label.value_counts()

In [22]:
from sklearn.preprocessing import  quantile_transform

In [23]:
li = ['Age','RestingBP','Cholesterol','MaxHR','Oldpeak']
for i in li:
    feature[i] = quantile_transform(feature[[i]],output_distribution='normal')

In [24]:
plt.figure(figsize=(15,15))
for i in enumerate(data.columns):
    plt.subplot(3,4,i[0]+1)
    sns.distplot(data[i[1]])
    

In [25]:
from sklearn.model_selection import train_test_split
trainF,testF,trainL,testL = train_test_split(feature,label,random_state=42,test_size=.2)

In [26]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(trainF,trainL)
pred = model.predict(testF)
from sklearn.metrics import classification_report
print(classification_report(pred,testL))

# CrossValidation

In [28]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[x**2 for x in range(3,10)],
    'n_jobs':[-1,None],
    "random_state":[42,None]    
}
cv = GridSearchCV(model,params,cv=5)
cv.fit(trainF,trainL)

In [ ]:
model = cv.best_estimator_ 

In [ ]:
model.fit(trainF,trainL)